In [1]:
# Tanawat Panitpongsri UID 7060636642

In [2]:
# HW 6 ECON425T

# Problem 1 (ii)

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats.outliers_influence as smo
import patsy as pt
import statsmodels.stats.api as sms
from simple_colors import *
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy.stats import truncnorm
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from scipy.optimize import minimize_scalar
from sklearn.linear_model import LassoCV, RidgeCV

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, auc
from sklearn import tree  
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import TensorDataset
from scipy.optimize import minimize

In [6]:
# Generate data
np.random.seed(42)
n_samples = 1000000
X = np.random.normal(0, 1, n_samples)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

y = sigmoid(X)

# Construct 1-layer NN
def f_nn1(weights, X):
    w0, w1 = weights
    h1 = sigmoid(w0 * X)
    y_hat_nn1 = w1 * h1
    return y_hat_nn1

# Construct 2-layer NN
def f_nn2(weights, X):
    w0, w1, w2 = weights
    h1 = sigmoid(w0 * X)
    h2 = sigmoid(w1 * h1)
    y_hat_nn2 = w2 * h2
    return y_hat_nn2

# Define the average squared loss function
def average_squared_loss(weights, X, y, nn_model):
    y_pred = nn_model(weights, X)
    return np.mean((y_pred - y) ** 2)

# Define a function for optimization using scipy.optimize.minimize
def optimize_nn(nn_model, X, y, initial_weights):
    result = minimize(fun=average_squared_loss, x0=initial_weights, args=(X, y, nn_model))
    return result.x, result.fun

# Initialize weights for a one-layer NN
initial_weights_nn1 = np.array([0.0, 0.0])

# Fit the one-layer NN
optimized_weights_nn1, training_error_nn1 = optimize_nn(f_nn1, X, y, initial_weights_nn1)

# Print the optimized weight and training error for the single-layer NN
print("Optimized weights for one-layer NN:", optimized_weights_nn1)
print("Training error for one-layer NN:", training_error_nn1)

# Initialize weights for the two-layer NN
initial_weights_nn2 = np.array([0.0, 0.0, 0.0])

# Fit the two-layer NN
optimized_weights_nn2, training_error_nn2 = optimize_nn(f_nn2, X, y, initial_weights_nn2)

# Print the optimized weights and training error for the two-layer NN
print("Optimized weights for two-layer NN:", optimized_weights_nn2)
print("Training error for two-layer NN:", training_error_nn2)

Optimized weights for one-layer NN: [0.99999486 1.        ]
Training error for one-layer NN: 6.27678327139154e-13
Optimized weights for two-layer NN: [5.59245113 3.46517549 0.67279059]
Training error for two-layer NN: 0.010079973099081412


## (ii)

From one-layer NN, I get w0 and w1 value closing to 1 and get training error close to 0.
On the other hand, I get higher training error from two-layer NN compared to the one-layer NN. I also get w0 = 5.59245113, w1 = 3.46517549, w2 = 0.67279059.
The two-layer NN is too complicated for the true relationship of y and x. This makes trainging error of two-layer NN is significant higher than training error of the one-layer NN, which is similar to the true relationship.

# Problem 2

In [7]:
df = pd.read_csv("card_transdata.csv")

In [8]:
y = df[["fraud"]].copy()
X = df.drop("fraud", axis = 1)

categorical_feature_names = ['repeat_retailer', 'used_chip', 'used_pin_number', 'online_order']
categorical_features_indices = [X.columns.get_loc(name) for name in categorical_feature_names]

In [9]:
# Convert DataFrames to numpy arrays
X_train_numpy = X[:500000].to_numpy(dtype=np.float32)
y_train_numpy = y[:500000].to_numpy(dtype=np.float32).ravel()
X_test_numpy = X[500000:].to_numpy(dtype=np.float32)
y_test_numpy = y[500000:].to_numpy(dtype=np.float32).ravel()

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train_numpy)
y_train_tensor = torch.from_numpy(y_train_numpy).long()  # Ensure target tensor is of type long
X_test_tensor = torch.from_numpy(X_test_numpy)
y_test_tensor = torch.from_numpy(y_test_numpy).long()

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Define DataLoader
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [10]:
# Define the Neural Network model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(X_train_numpy.shape[1], 512),  # Adjust the input features dynamically
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, len(np.unique(y_train_numpy)))  # Adjust the output features dynamically
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# Instantiate the model
model = NeuralNetwork()

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [12]:
# Training function
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{len(dataloader.dataset):>5d}]")


In [13]:
# Testing function
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [14]:
# Run training and testing
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.734922  [    0/500000]
loss: 0.514990  [ 6400/500000]
loss: 1.408822  [12800/500000]
loss: 0.509477  [19200/500000]
loss: 0.652709  [25600/500000]
loss: 0.388789  [32000/500000]
loss: 0.384087  [38400/500000]
loss: 0.387976  [44800/500000]
loss: 0.380724  [51200/500000]
loss: 0.531155  [57600/500000]
loss: 0.265888  [64000/500000]
loss: 0.362437  [70400/500000]
loss: 0.407665  [76800/500000]
loss: 0.287708  [83200/500000]
loss: 0.338064  [89600/500000]
loss: 0.292732  [96000/500000]
loss: 0.142228  [102400/500000]
loss: 0.336705  [108800/500000]
loss: 0.220611  [115200/500000]
loss: 0.240624  [121600/500000]
loss: 0.558213  [128000/500000]
loss: 0.201503  [134400/500000]
loss: 0.283654  [140800/500000]
loss: 0.298639  [147200/500000]
loss: 0.310254  [153600/500000]
loss: 0.196349  [160000/500000]
loss: 0.726584  [166400/500000]
loss: 0.192304  [172800/500000]
loss: 0.200646  [179200/500000]
loss: 0.186326  [185600/500000]
loss: 0.275377  

loss: 0.111932  [76800/500000]
loss: 0.107608  [83200/500000]
loss: 0.282060  [89600/500000]
loss: 0.140874  [96000/500000]
loss: 0.035154  [102400/500000]
loss: 0.129785  [108800/500000]
loss: 0.119486  [115200/500000]
loss: 0.126072  [121600/500000]
loss: 0.306278  [128000/500000]
loss: 0.074841  [134400/500000]
loss: 0.156402  [140800/500000]
loss: 0.137816  [147200/500000]
loss: 0.066412  [153600/500000]
loss: 0.100588  [160000/500000]
loss: 0.086298  [166400/500000]
loss: 0.082366  [172800/500000]
loss: 0.088186  [179200/500000]
loss: 0.065993  [185600/500000]
loss: 0.153835  [192000/500000]
loss: 0.046290  [198400/500000]
loss: 0.070381  [204800/500000]
loss: 0.104790  [211200/500000]
loss: 0.158993  [217600/500000]
loss: 0.122310  [224000/500000]
loss: 0.191072  [230400/500000]
loss: 0.133120  [236800/500000]
loss: 0.060792  [243200/500000]
loss: 0.147721  [249600/500000]
loss: 0.123170  [256000/500000]
loss: 0.044291  [262400/500000]
loss: 0.269368  [268800/500000]
loss: 0.1306

In [19]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def test(dataloader, model):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            all_preds.extend(pred.argmax(1).cpu().numpy())  # Append predicted labels
            all_labels.extend(y.cpu().numpy())  # Append true labels
    
    # Calculate metrics
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')
    accuracy = accuracy_score(all_labels, all_preds)  # Calculate accuracy
    
    # Print metrics
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1: {f1:.4f}")

test(test_dataloader, model)

Test Accuracy: 0.9652
Test Precision: 0.8801
Test Recall: 0.6967
Test F1: 0.7777


From the trained model performance, I get the final model with accuracy = 96.5% improving from 92.3% from the first round of iterations while the average loss decreses from 0.1803 to 0.0891 in the final iteration.
However, the simple decision tree from Homework5 outperform this NN model since the decision tree model has Recall = 99.97%, Precision = 100%, and F1 = 99.99%. The NN model has Test Recall = 69.67%, Precision = 88.01%, and F1 = 77.77%.